In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import geopandas as gpd

### VMT comparison by SuperDistrict and TAZ

In [2]:
# read STIP Project run resident vmt output
vmt_Proj = pd.read_csv(r'M:\Application\Model One\STIP2022\2050_TM152_STP_Project_00\OUTPUT\core_summaries\VehicleMilesTraveled.csv')
# print(list(vmt_Proj))
vmt_Proj['vmt_tot'] = vmt_Proj['freq'] * vmt_Proj['vmt']

# summarize by SD and TAZ
vmt_taz_Proj = vmt_Proj.groupby(['county_name', 'SD', 'taz'])['vmt_tot'].sum().reset_index()

In [3]:
# read STIP NoProject run resident vmt output
vmt_noProj = pd.read_csv(r'M:\Application\Model One\STIP2022\2050_TM152_STP_NoProject_00\OUTPUT\core_summaries\VehicleMilesTraveled.csv')
# print(list(vmt_noProj))
vmt_noProj['vmt_tot'] = vmt_noProj['freq'] * vmt_noProj['vmt']

# summarize by SD and TAZ
vmt_taz_noProj = vmt_noProj.groupby(['county_name', 'SD', 'taz'])['vmt_tot'].sum().reset_index() 

In [4]:
# read Final Blueprint run resident vmt output
vmt_FBP = pd.read_csv(r'M:\Application\Model One\RTP2021\Blueprint\2050_TM152_FBP_PlusCrossing_24\OUTPUT\core_summaries\VehicleMilesTraveled.csv')
# print(list(vmt_FBP))
vmt_FBP['vmt_tot'] = vmt_FBP['freq'] * vmt_FBP['vmt']

# summarize by SD and TAZ
vmt_taz_FBP = vmt_FBP.groupby(['county_name', 'SD', 'taz'])['vmt_tot'].sum().reset_index() 

In [5]:
# check totals
print(vmt_Proj.vmt_tot.sum())
print(vmt_noProj.vmt_tot.sum())
print(vmt_FBP.vmt_tot.sum())

142274252.9376923
142360376.9853846
122735250.07769233


In [6]:
# label run and merge

vmt_taz_Proj.rename(columns = {'vmt_tot': 'vmt_Proj'}, inplace=True)
vmt_taz_noProj.rename(columns = {'vmt_tot': 'vmt_noProj'}, inplace=True)
vmt_taz_FBP.rename(columns = {'vmt_tot': 'vmt_FBP'}, inplace=True)

vmt_comp = vmt_taz_Proj.merge(vmt_taz_noProj,
                              on=['county_name', 'SD', 'taz'],
                              how='outer').merge(vmt_taz_FBP,
                                                 on=['county_name', 'SD', 'taz'],
                                                 how='outer')

In [7]:
# export for Tableau
vmt_comp.to_csv(r'M:\Application\Model One\STIP2022\VMT_QAQC\vmt_by_TAZ_comparsion.csv', index=False)

### VMT comparison on Highway Networks

In [8]:
# read Project run loaded network
proj_raw = gpd.read_file(r'M:\Application\Model One\STIP2022\2050_TM152_STP_Project_00\shapefile\network_links.shp')
print('Project highway networks has {} links'.format(proj_raw.shape[0]))
proj = proj_raw[['A', 'B', 'VMT24HR', 'geometry', 'PROJ']]
proj.rename(columns = {'VMT24HR': 'VMT_proj',
                       'geometry': 'geometry_proj'}, inplace=True)

Project highway networks has 33978 links


In [9]:
print(proj_raw.crs)

epsg:26910


In [10]:
# read NoProject run loaded network
noProj_raw = gpd.read_file(r'M:\Application\Model One\STIP2022\2050_TM152_STP_NoProject_00\shapefile\network_links.shp')
print('NoProject highway networks has {} links'.format(noProj_raw.shape[0]))
noProj = noProj_raw[['A', 'B', 'VMT24HR', 'geometry']]
noProj.rename(columns = {'VMT24HR': 'VMT_noProj',
                         'geometry': 'geometry_noProj'}, inplace=True)

NoProject highway networks has 33937 links


In [11]:
# merge
vmt_network_comp = proj.merge(noProj,
                              on=['A', 'B'],
                              how='outer')
print('merged highway networks has {} links'.format(vmt_network_comp.shape[0]))

merged highway networks has 33981 links


In [12]:
# use Project geometry; for links without Project geometry (deleted by projects), use noProj geometry 
vmt_network_comp['geometry'] = vmt_network_comp['geometry_proj']
vmt_network_comp.loc[vmt_network_comp.geometry_proj.isnull(), 'geometry'] = vmt_network_comp['geometry_noProj']

In [13]:
# convert to geodataframe
vmt_network_comp_gpd = gpd.GeoDataFrame(vmt_network_comp, crs="EPSG:26910", geometry='geometry')

In [14]:
# export for Tableau
vmt_network_comp_gpd[['A', 'B', 'VMT_proj', 'VMT_noProj', 'PROJ', 'geometry'
                     ]].to_file(r'M:\Application\Model One\STIP2022\VMT_QAQC\vmt_hwylinks_comparsion.shp')